<a href="https://colab.research.google.com/github/gcsilva83/ChatGPT_Notebooks/blob/main/Run_streamlit_app_from_a_Google_Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run streamlit app from a Google Colab Notebook
> Created by [Manuel Romero](https://twitter.com/mrm8488)

In [1]:
!pip install -q streamlit

In [2]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.125.44.53


In [3]:
%%writefile .env
OPENAI_API_KEY=sk-

Overwriting .env


## Create a streamlit app example


In [4]:
%%writefile app.py
from langchain.agents import create_csv_agent,initialize_agent, Tool
#from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.callbacks.streaming_stdout_final_only import FinalStreamingStdOutCallbackHandler
from langchain.callbacks.base import AsyncCallbackHandler
from IPython.display import Markdown, display
from timeit import default_timer as timer
import json
# "/Users/metro/Documents/Data & ChatGPT/gpt/bin/activate"

from dotenv import load_dotenv
from tempfile import NamedTemporaryFile

from PIL import Image
import streamlit as st
import pandas as pd
import numpy as np
import os

def main():
    load_dotenv()

    # Load the OpenAI API key from the environment variable
    openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key:
        raise ValueError("OPENAI_API_KEY is not set")

    #st.set_page_config(page_title="Talk with your CSV 📊")
    st.set_page_config(
        page_title="Talk with your CSV 📊",
        page_icon=":bar_chart:",
        layout="wide",
        initial_sidebar_state="collapsed",
)
    st.header("Start Ask to your CSV 📈🦅")
    st.subheader('Make a question and let ChatGPT answer it for ya!! :sunglasses:')
    #st.caption('A caption with _italics_ :blue[colors] and emojis :sunglasses:')
    #logo = Image.open("header_Icon_sidebar.jpg")
    # https://ibb.co/7QPd3xD
    #st.sidebar.image(logo, use_column_width=True)
    # Using "with" notation
    with st.sidebar:
        "## This is the sidebar"


        add_radio = st.radio(
           "Select a Model",
            ("gpt-3.5-turbo", "text-davinci-003","GPT-4")

        )


        st.slider("Define the Temperature to be applied into the Agent model",min_value=0.0, max_value=1.0, value=0.0, step=0.1)
        #st.markdown("The next word is <span style='color:red'>red</span>", unsafe_allow_html=True)
        st.caption("**:black[_One intriguing parameter within LLMs is the “temperature.” The LLM temperature is a hyperparameter that regulates the randomness, or creativity, of the AI’s responses. A higher temperature value typically makes the output more diverse and creative but might also increase its likelihood of straying from the context. Conversely, a lower temperature value makes the AI’s responses more focused and deterministic, sticking closely to the most likely prediction._]**")
        # Using object notation
        add_selectbox = st.sidebar.selectbox( "Which Agent would you like to use?",("CSV", "Pandas","Python")
    )
    st.markdown("""
    <style>
        [data-testid=stSidebar] {
            background-color: #17FFE7 ;
        }
    </style>
    """, unsafe_allow_html=True)


    # Define the variables
    max_width = 1400
    padding_top = 0.8
    padding_right = 0.8
    padding_left = 0.8
    padding_bottom = 0.8
    COLOR = "#339933"
    BACKGROUND_COLOR = "#F0F9F6"

    st.caption(f'You have select _{add_radio}_ to run within your ChatBot!! :sunglasses:')
# Use the variables in the st.markdown code snippet
    st.markdown(
        f"""
        <style>
            .reportview-container .main .block-container {{
                max-width: {max_width}px;
                padding-top: {padding_top}rem;
                padding-right: {padding_right}rem;
                padding-left: {padding_left}rem;
                padding-bottom: {padding_bottom}rem;
            }}
            .reportview-container .main {{
                color: {COLOR};
                background-color: {BACKGROUND_COLOR};
            }}
        </style>
        """,
        unsafe_allow_html=True,
    )

    user_csv = st.file_uploader("Please, upload your CSV file", type="csv")

    if user_csv:
        with NamedTemporaryFile() as f:
            f.write(user_csv.getvalue())
            if add_radio == "text-davinci-003":
                llm = OpenAI(model_name="text-davinci-003", temperature=0)
            elif add_radio == "GPT-4" :
                llm = OpenAI( temperature=0)
            else:
                llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)
            #llm = OpenAI(model_name=""text-ada-001"", temperature=0)
            #llm = OpenAI( temperature=0)
            user_input = st.text_input("Question here:")

            agent = create_csv_agent(llm, f.name, verbose=True)

            with get_openai_callback() as cb:
                if user_input:
                    with st.spinner(text="In progress..."):
                        start = timer()
                        output = agent.run(user_input)
                        st.write(output)
                        total_tokens = f"Total Tokens: {cb.total_tokens}"
                        prompt_tokens = f"Prompt Tokens: {cb.prompt_tokens}"
                        completion_tokens = f"Completion Tokens: {cb.completion_tokens}"
                        total_cost = f"Total Cost (USD): ${cb.total_cost:.5f}"
                        end = timer()
                        calc = end - start
                        with st.sidebar:
                            with st.expander("Some stats about the AgentExecution"):
                                st.markdown(f"Total Tokens: <span style='color:black'>{cb.total_tokens}</span>", unsafe_allow_html=True)
                                st.markdown(f"Prompt Tokens:  <span style='color:black'>{cb.prompt_tokens}</span>", unsafe_allow_html=True)
                                st.markdown(f"Completion Tokens:  <span style='color:black'>{cb.completion_tokens}</span>", unsafe_allow_html=True)
                                st.markdown(f"Total Cost (USD): $ <span style='color:black'>{cb.total_cost:.5f}</span>", unsafe_allow_html=True)
                                st.markdown(f"Runtime of the program was <span style='color:black'>{calc:.2f} seconds</span> seconds", unsafe_allow_html=True)

                        with st.expander("See explanation"):
                            st.write(agent.agent.llm_chain.prompt.template)
                       ## with st.expander("See explanation_"):
                       ##     st.write(agent.agent)
                       ## with st.expander("See explanation__"):
                       ##     st.write(agent.agent)


if __name__ == "__main__":
    main()


Overwriting app.py


## Install localtunnel

In [5]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 1.22s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



## Run streamlit in background

In [6]:
!streamlit run /content/app.py &>/content/logs.txt &

In [7]:
pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Expose the port 8501
Then just click in the `url` showed.

A `log.txt`file will be created.

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.83s
your url is: https://sixty-groups-vanish.loca.lt


[![ko-fi](https://www.ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/Y8Y3VYYE)